<img src="images/steno3d.png" width="200"></img>

<br>

# Comprehensive Example: Wolf Pass Exploration Project

In this notebook, we will use <a href="https://steno3d.com">Steno3D</a> to view and share data throughout the steps of a mineral resource exploration project at Wolf Pass. The steps in this project include:

- <a href="#Inspect-the-Surface-Topography">Inspecting the Surface Topography</a>
- <a href="#Decide-on-Drill-Locations">Deciding on Drill Locations</a>
- <a href="#Plot-the-Borehole-Data">Plotting the Borehole Data</a>
- <a href="#Explore-the-Modeled-Formation">Exploring the Modeled Formation</a>

___
**Navigation**
- <a href="index.ipynb">Notebook home</a>
- <a href="https://steno3d.com">Steno3D website</a>
- <a href="https://steno3d.com/docs">Steno3D documentation</a>
- <a href="https://github.com/seequent/steno3d-notebooks/issues/new">Report an issue</a>
___

In [3]:
import steno3d

## Login to Steno3D

The first thing you need to do is login to Steno3D. You can <a href="https://steno3d.com/signup">sign up for an account</a> to get your own developer API key if you do not have one already. Running the cell below will provide you with instructions for how to obtain and enter your key.

In [4]:
steno3d.login()


Oh no! Your version of steno3d is out of date.

Your version: 0.3.11
Current version: 0.3.10

Please update steno3d with `pip install --upgrade steno3d`.



>> Welcome to the Python client library for Steno3D!

Credentials file found: /Users/mmorley/.steno3d_client/credentials
Accessing API developer key for @mmorley
Welcome to Steno3D! You are logged in as @mmorley


For this notebook, we will be using example data from the `steno3d.examples.Wolfpass` module. This module downloads raw data, saves it to a folder in the home directory (or another location you may specify), and allows easy access to all the different pieces of data.

In [6]:
from steno3d.examples import Wolfpass
Wolfpass.fetch_data(verbose=False)  # To specify a different data directory than the
                                    # default '$HOME/.steno3d_python_assets/' use
                                    # keyword argument `directory='/path/to/your/directory/'`

## Inspect the Surface Topography

The first step of using Steno3D is to create a <a href="https://python.steno3d.com/en/latest/content/api/projects.html">project</a>. Let's make one that will include all our preliminary data used to plan the mineral resource exploration. We will create a different project later when we have some results to display.

This project is public, so anyone can view it once it is uploaded. When working with your own data, you may want to create private projects to share only with your collaborators. Different Steno3D accounts allow different numbers of private projects; <a href="https://steno3d.com/pricing">learn about these accounts and how to upgrade on the Steno3D website</a>.

In [5]:
prelim_data = steno3d.Project(
    title='Test',
    description='This project will contain surface topography and planned borehole drill locations.',
    public=True
)

Now we can start to construct our ground surface. To build the <a href="https://python.steno3d.com/en/latest/content/api/resources/surface.html#meshes">mesh</a>, we use vertices and triangles imported from our `Wolfpass` example module.

In [13]:
ground_mesh = steno3d.Mesh2D(
    vertices=Wolfpass.topo_vertices,
    triangles=Wolfpass.topo_triangles
)

Elevation data will be useful; this can come directly from the `vertices`.

In [14]:
ground_elev = steno3d.DataArray(
    title='Ground Surface Elevation',
    description='Elevation values correspond to mesh vertices',
    array=Wolfpass.topo_vertices[:,2]  # Elevation corresponds to Z-component of vertices
)

We also have some surface imagery of the area. The image and image orientation are avilable in the `Wolfpass` example module. They can be applied as a <a href="https://python.steno3d.com/en/latest/content/api/resources/texture.html#resources-texture">texture</a>.

In [15]:
ground_image = steno3d.Texture2DImage(
    title='Surface Imagery',
    image=Wolfpass.topo_image,
    O=Wolfpass.topo_image_orientation['O'],
    U=Wolfpass.topo_image_orientation['U'],
    V=Wolfpass.topo_image_orientation['V']
)

We can now add these components to our ground surface. (For tips on the data binding step, refer to <a href="https://python.steno3d.com/en/latest/content/api/resources/surface.html#data">binding data to surface</a>.

In [16]:
ground_surf = steno3d.Surface(
    project=prelim_data,
    title='Ground Surface',
    description='Surface with elevation data and surface imagery.',
    mesh=ground_mesh,
    textures=ground_image,
    data=[dict(location='N',
              data=ground_elev)]
)

We can now upload this surface to <a href="https://steno3d.com">steno3d.com</a> and view the surface here in the notebook to ensure it looks okay.

In [17]:
ground_surf.upload()
ground_surf.plot()

Verifying your quota for public projects...
This PUBLIC project will be viewable by everyone.
Total progress: 100% - Uploading: project Testnd Surfacelevation
Complete!
https://steno3d.com/resource/surface/SB40EyfEzydBaA7FS2fW
Resource not uploaded: Please upload() before plotting.


## Decide on Drill Locations

Now that we have the ground surface, we need to add some proposed drill location points. First, we need a <a href="https://python.steno3d.com/en/latest/content/api/resources/point.html#meshes">mesh 0D</a>. The vertex data is imported from the `Wolfpass` module.

In [10]:
drill_locations = steno3d.Mesh0D(
    vertices=Wolfpass.drill_vertices
)

In [11]:
print(Wolfpass.drill_vertices)

[[445198.219      494110.59399707   3057.75753788]
 [445196.62499823 494109.65600879   3054.2654076 ]
 [445260.56299986 493860.71899707   3009.03832773]
 [445203.344      493720.65600116   3033.77766423]
 [445291.56300177 493566.21900089   2957.43473057]
 [445294.03100027 493886.81299121   2970.84867672]
 [445296.594      493890.25000293   2974.07976727]
 [445460.81299414 493321.15600879   2979.14362339]
 [445452.31299414 493402.18800647   2980.30098728]
 [445427.219      493523.68799884   2978.65771488]
 [445149.15599646 493884.59399707   3099.24893956]
 [445293.844      493889.87500293   2973.70386128]
 [445536.15601172 493143.53100879   2976.94469265]
 [445633.219      492890.34399707   2935.5617371 ]
 [445203.12499823 493720.18799884   3033.79004673]
 [445567.37500586 493500.90600116   2885.07222752]
 [445455.094      493658.18799884   2874.97363285]
 [445203.74999823 493719.78100116   3033.17045788]
 [445708.62500586 493127.18799121   2880.63351445]
 [445294.65600027 493888.093997

We only need the locations of these points, no other data, so let's construct our <a href="https://python.steno3d.com/en/latest/content/api/resources/point.html">point</a> resource now. We will make the points orange so they stand out when plotted with the surface imagery.

In [19]:
drill_points = steno3d.Point(
    project=prelim_data,
    title='Proposed Drill Locations',
    mesh=drill_locations,
    opts=dict(color='orange')
)

After uploading these points to our project on <a href="https://steno3d.com">steno3d.com</a>, let's print the url of our project to share.

In [20]:
drill_points.upload()

This project is PUBLIC. It is viewable by everyone.
Total progress:  18% - Uploading: project Tested Drill Locations
Complete!
https://steno3d.com/resource/point/hnDAVyyMDWmsyueJXby5


'https://steno3d.com/resource/point/hnDAVyyMDWmsyueJXby5'

In [21]:
print(prelim_data.url)

https://steno3d.com/app/mAflLQZ1R6wsvOeiUesE


## Plot the Borehole Data

After discussing the preliminary data project online with collaborators, it was decided to go ahead with borehole drilling. Now the data is back and available to be explored with. Let's make a new project to contain the results of the drilling.

In [12]:
result_data = steno3d.Project(
    title='Test 2: Results',
    description='This project will contain borehole data and interpreted formation surface.',
    public=True  # Public for our example, since private quota is limited.
)

To display our borehole data, we first need a <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html#meshes">mesh 1D</a> based on their spatial geometry. This is imported from the `Wolfpass` module. The `view_type` option sets the lines mesh to initially display as fixed-width, extruded "tubes" rather than true 1D lines.

In [7]:
print(Wolfpass.borehole_vertices)
print( Wolfpass.borehole_segments)

borehole_mesh = steno3d.Mesh1D(
    vertices = Wolfpass.borehole_vertices,
    segments = Wolfpass.borehole_segments,
    opts = dict(
        view_type='boreholes'
    )
)

[[445198.219      494110.594        3057.75753665]
 [445199.50452627 494110.58278138   3056.22544776]
 [445200.79005254 494110.57156276   3054.69335888]
 ...
 [445465.11394538 493600.           2825.50671284]
 [445464.42990509 493600.           2823.6273276 ]
 [445463.93397691 493600.           2822.26477612]]
[[   0    1]
 [   1    2]
 [   2    3]
 ...
 [8575 8576]
 [8576 8577]
 [8577 8578]]


Let's create a <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html">line</a> resource now using this mesh.

In [16]:
boreholes = steno3d.Line(
    project=result_data,
    title='Borehole Results',
    description='Data based on results of borehole drilling',
    mesh=borehole_mesh
)

We need to add all the data to the boreholes now.

In [17]:
borehole_raw_data = Wolfpass.borehole_data
print (Wolfpass.borehole_data)

borehole_data = [
    steno3d.DataArray(title=rd, array=borehole_raw_data[rd]) for rd in borehole_raw_data
]
borehole_bound_data = [
    dict(location='CC', data=d) for d in borehole_data
]

{'AG_gpt': array([6.35, 5.2 , 5.75, ...,  nan,  nan,  nan]), 'AS_ppm': array([26.3, 31. , 32.5, ...,  nan,  nan,  nan]), 'AU_gpt': array([1.75, 1.73, 6.  , ..., 0.38, 0.65, 0.66]), 'CU_pct': array([0.79, 0.83, 0.84, ...,  nan,  nan,  nan]), 'Density': array([ nan,  nan,  nan, ..., 2.93, 2.81, 2.97]), 'MO_ppm': array([10. , 12.2, 24.8, ...,  nan,  nan,  nan]), 'Recov': array([0.5 , 1.3 , 1.8 , ..., 2.  , 1.8 , 1.45]), 'S_pct': array([ 0.,  0.,  0., ..., nan, nan, nan]), 'dacite_data': array([0.05489251, 0.05580048, 0.05672582, ..., 0.27216602, 0.26993653,
       0.2657913 ])}


In [26]:
boreholes.data = borehole_bound_data

These boreholes will be easier to analyze if the ground surface is also present. We can simply add the ground surface resource created for the <a href="#Inspect-the-Surface-Topography">preliminary data project</a> to our results data project.

In [27]:
result_data.resources += [ground_surf]

We can adjust the opacity of this surface so the boreholes are not obscured.

In [28]:
ground_surf.opts.opacity = 0.25

Now our borehole results can be uploaded and shared.

In [29]:
result_data.upload()

Verifying your quota for public projects...
This PUBLIC project will be viewable by everyone.
Total progress:  23% - Uploading: project Test 2: Resultsevation
Complete!
https://steno3d.com/app/j18YH5DCbSP772AjOmop


'j18YH5DCbSP772AjOmop'

## Explore the Modeled Formation

Based on the borehole results, modelling of the dacite formation has been carried out. Let's view this modeled surface with our other results using <a href="https://steno3d.com">Steno3D</a>. We will create, upload, and print the url for sharing in one step.

In [30]:
dacite_ind = Wolfpass.lith_names.index('dacite')
dacite_vertices = Wolfpass.lith_vertices[dacite_ind]
dacite_triangles = Wolfpass.lith_triangles[dacite_ind]

dacite_surface = steno3d.Surface(
    project=result_data,
    title='Dacite Formation',
    description='Bounding surface of dacite based on borehole data and modelling results',
    mesh=steno3d.Mesh2D(
        vertices=dacite_vertices,
        triangles=dacite_triangles
    ),
    opts=dict(
        color='red'
    )
)
dacite_surface.upload()
print(result_data.url)

This project is PUBLIC. It is viewable by everyone.
Total progress:  27% - Uploading: project Test 2: Resultsnvation
Complete!
https://steno3d.com/resource/surface/CZrqBU2o2oYEPlDEmbEW
https://steno3d.com/app/j18YH5DCbSP772AjOmop


___
**Navigation**
- <a href="#Comprehensive-Example:-Wolf-Pass-Exploration-Project">Top of page</a>
- <a href="index.ipynb">Notebook home</a>
- <a href="https://steno3d.com">Steno3D website</a>
- <a href="https://steno3d.com/docs">Steno3D documentation</a>
- <a href="https://github.com/seequent/steno3d-notebooks/issues/new">Report an issue</a>
___